In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

#Matheus do Ó Santos Tiburcio

#Projeto 1 de Estatística e Probabilidade (MAD243)

#a)

  O problema consiste em um jogador que, normalmente, tende a trocar de casas, mas pelas descrições sabemos que não importa onde ele já esteve, mas somente onde ele atualmente está. Por ser um jogo baseado em rolagem de dados definindo a casa para que se vai, somente a casa que atualmente se está interessa, pois essa será somada ao valor dos dados, levando o jogador a uma nova casa. E essa é justamente a ideia das Cadeias de Markov, pois nelas não nos interessa todo o passado, mas somente o estado atual:

P(X = x | Y = y) = P(X = x | Y = y; Z = z; .... W = w)

  Nessa matriz que chamaremos de matriz de transição cada linha representaria a casa em que o jogador atualmente está e as colunas conterão as probabilidades de o jogador cair naquela casa especificada pelo valor da coluna. Desse modo se olharmos a linha 1, coluna 5, vamos estar vendo a probabilidade de se cair na casa 5 sabendo que estamos na casa 1. Por esse exemplo fica um pouco mais plausível o uso de uma cadeia de Markov.



---



Agora as probabilidades de em dada casa, cair noutra:

  Antes vamos organizar o que temos, sabemos que 2 dados são rolados e que caso sejam iguais o jogador não se move, sendo assim:

  Temos que dentre os 36 casos possíveis temos que 6 terão dados iguais, logo:

P(dados iguais) = 6/36

Para os outros valores da soma de dados, observamos que:

S = soma dos dados, P(S):

P(1) = 0

P(2) = 1/36

P(3) = 2/36

P(4) = 3/36

P(5) = 4/36

P(6) = 5/36

P(7) = 6/36

P(8) = 5/36

P(9) = 4/36

P(10) = 3/36

P(11) = 2/36

P(12) = 1/36

Porém casos onde os dados são iguais são desconsiderados, logo:

P(1) = 0

P(2) = 0

P(3) = 2/36

P(4) = 2/36

P(5) = 4/36

P(6) = 4/36

P(7) = 6/36

P(8) = 4/36

P(9) = 4/36

P(10) = 2/36

P(11) = 2/36

P(dados iguais) = 6/36



---



  Portanto a matriz de transição irá conter essa informação. No caso de a chance de S = 3 ser 2/36, significaria, por exemplo, que se estou na casa 1, tenho 2/36 de chance de cair na casa 4 (1 + 3).

  Devido a regra de ir à cadeia caso se tire números iguais 3 vezes seguidas, eu utilizei estados auxiliares. Os estados então seriam estados normais, estados onde já se rolou números iguais uma vez e duas vezes. Então teríamos por exemplo: 1 1' e 1'' para respectivamente "estou na casa 1", "estou na casa 1 e rolei dados iguais 1 vez" e "estou na casa 1 e rolei dados iguais 2 vezes seguidas". Caso se role uma terceira, se vai a um outro estado especial que representa o fato de você estar preso.

  Quando se está preso há 2 modos de sair: ficando lá por três turnos ou tirando números iguais nos dados. Para modelar isso usei mais estados auxiliares(tem muitos!) sendo preso, preso' e preso''. A lógica aqui é parecida, mas eles representariam respectivamente: "estou preso", "estou preso e NÃO tirei dados iguais" e "estou preso e NÃO tirei dados iguais duas vezes seguidas". Caso se esteja em preso'', automaticamente você é solto da prisão e o comportamento da linha é igual ao de estar na casa 20. Mas quais as probabilidades disso? Bom, se queremos dados iguais, a chance de transitar de preso para preso' e de preso' para preso'' é não tirar dados iguais. Essa chance é de 1 - chance de tirar, logo: 1 - 6/36 = 30/36. E a chance de sair da prisão E rolar um dado normalmente é (6/36) * probalidade de se cair S valor na soma dos dados.

  O código abaixo monta essa matriz de transição, junto de um código que permite acessar uma linha ou coluna em específico dessa matriz.

In [ ]:
def DefineEstocastica():
  probabilidadesDado = [0, 0, 0, 2/36, 2/36, 4/36,
                        4/36, 6/36, 4/36, 4/36, 2/36, 2/36, 0]

  M = np.zeros((123, 123))
  j = 0
  totalCol = 0
  probLinha = 0

  #percorre as linhas
  for i in range(123):
    #percorre as colunas
    while(totalCol <= 122):
      #o percurso começa exatamente no valor de entrada i, i
      if(totalCol == 0):
        if(i < 40):
          j = i
        elif(i >= 40 and i <= 79):
          j = i - 40
        elif(i >= 80 and i <= 119):
          j = i - 80
        else:
          j = 19
        #percorre todo os possiveis caminhos com os dados jogados
        #com valores diferentes
        for p in probabilidadesDado:
          if(i < 120 or i == 122):
            M[i, (j + totalCol) % 40] = p
          else:
            M[i, (j + totalCol) % 40] = (6/36)*p
          totalCol += 1
        j = (j + totalCol) % 123

      #calcula a probabilidade de linha (dados repetidos)
      elif(i >= 0 and i <= 79 and probLinha == 0):
        M[i, i + 40] = 6/36
        probLinha = 1
        totalCol += 1
      elif(i >= 80 and i <= 119 and probLinha == 0):
        M[i, 120] = 6/36
        probLinha = 1
        totalCol += 1
      elif(i == 120 and probLinha == 0):
        M[i, 120] = (6/36)**2
        M[i, 121] = 30/36
        probLinha = 1
        totalCol += 1
      elif(i == 121 and probLinha == 0):
        M[i, 120] = (6/36)**2
        M[i, 122] = 30/36
        probLinha = 1
        totalCol += 1
      elif(i == 122 and probLinha == 0):
        M[i, 59] = 6/36
        probLinha = 1
        totalCol += 1
      #só uma checagem para não sobrepor o valor
      #de dados repetidos feito acima
      elif(M[i, j] != 6/36):
        M[i, j] = 0
        totalCol += 1

      j = (j + 1) % 123
      probLinha = 0
    totalCol = 0
  return M

In [ ]:
M = DefineEstocastica()

In [ ]:
def AcessaLinhaColuna(c, i, M):
  if(c == 0):
    return M[i, :]
  else:
    return M[:, i]

A função que forma a matriz estocástica é confusa, tentei comentar no código, mas não sei se ajudou muito, desculpe. Já a segunda é só para uma checagem de linha ou coluna. c é o comando para saber se se quer linha ou coluna. Se c for 0, retorna-se a linha, se for diferente de 0, retorna coluna. i é o indice da linha ou coluna e M a matriz.

Na verdade acho que usar M[i, :] e M[:, i] para acessar essas linhas e colunas mais rápido e menos "trabalhoso".


---


#b)

Como o jogo depende de rolagem de dados, vou fazer uma função de soma de dados para auxiliar o processo:

In [ ]:
#n é o número de dados arremessados ou de arremessos
def ArremessaDados(n):
  arremesso = np.random.randint(1, 7, n)
  return arremesso

Em seguida temos uma função que simula uma partida com n iterações.

IMPORTANTE: Aqui eu diferencio turnos de iterações, turnos só mudam quando os dados tem valores distintos!

In [ ]:
def Joga(n):
  #X indica a casa que se está - 1
  X = 0
  #numeroRepetido representa quantas vezes seguidas se tirou números
  #repetidos nos dados
  numeroRepetido = 0
  #preso indica quantos turnos se está preso. Se for igual a 0
  #significa que o jogador não está preso
  preso = 0
  turno = 0
  for i in range(n):
    rolagemDeDados = ArremessaDados(2)

    if(preso != 0):
      preso += 1
      turno += 1
    if(preso == 3):
      preso = 0
      print("Foi solto!")

    if(preso < 3 and preso != 0):
      preso += 1
    else:
      preso = 0

    if(rolagemDeDados[0] != rolagemDeDados[1] and preso == 0):
      #move X em dado 1 + dado 2 casas
      X = (X + rolagemDeDados@np.ones(2)) % 40
      #reseta numeroRepetido
      numeroRepetido = 0
      turno += 1

    elif(preso > 0 and rolagemDeDados[0] == rolagemDeDados[1]):
      preso = 0
      numeroRepetido = 0
      print("Foi solto!")
    else:
      numeroRepetido += 1
      if(numeroRepetido == 3):
        numeroRepetido = 0
        preso = 1
        X = 19
        print("Está preso!")
    print(f"Turno {turno}: X = {X+1} | Rolagens repetidas seguidas: {numeroRepetido} | {rolagemDeDados}")

In [ ]:
Joga(100)

Turno 1: X = 8.0 | Rolagens repetidas seguidas: 0 | [6 1]
Turno 2: X = 17.0 | Rolagens repetidas seguidas: 0 | [4 5]
Turno 3: X = 22.0 | Rolagens repetidas seguidas: 0 | [1 4]
Turno 4: X = 32.0 | Rolagens repetidas seguidas: 0 | [4 6]
Turno 5: X = 36.0 | Rolagens repetidas seguidas: 0 | [3 1]
Turno 6: X = 5.0 | Rolagens repetidas seguidas: 0 | [4 5]
Turno 7: X = 9.0 | Rolagens repetidas seguidas: 0 | [3 1]
Turno 7: X = 9.0 | Rolagens repetidas seguidas: 1 | [4 4]
Turno 8: X = 13.0 | Rolagens repetidas seguidas: 0 | [1 3]
Turno 8: X = 13.0 | Rolagens repetidas seguidas: 1 | [4 4]
Turno 9: X = 16.0 | Rolagens repetidas seguidas: 0 | [1 2]
Turno 10: X = 21.0 | Rolagens repetidas seguidas: 0 | [2 3]
Turno 11: X = 26.0 | Rolagens repetidas seguidas: 0 | [1 4]
Turno 12: X = 33.0 | Rolagens repetidas seguidas: 0 | [4 3]
Turno 13: X = 2.0 | Rolagens repetidas seguidas: 0 | [4 5]
Turno 14: X = 9.0 | Rolagens repetidas seguidas: 0 | [1 6]
Turno 15: X = 15.0 | Rolagens repetidas seguidas: 0 | [4 

#c)

Aqui podemos tentar usar a Lei dos Grandes Números (https://pt.wikipedia.org/wiki/Lei_dos_grandes_números) que basicamente fala que a média aritmética dos resultados de um mesmo experimento repetido diversas vezes se aproxima de seu valor esperado para um *n* suficientemente grande.

Para resolver isso, temos uma função que simula uma partida até que você seja preso, quando preso ela retornará o turno em que se foi preso. Após isso vamos fazer esse experimento de ser preso repetidas vezes, somar os turnos em que isso ocorre e dividir pelo número de testes. A função basicamente é uma adaptação da função Joga(n) de cima.

In [ ]:
def JogaAteSerPreso():
  turno = 0
  #numeroRepetido representa quantas vezes seguidas se tirou números
  #repetidos nos dados
  numeroRepetido = 0

  while True:
    #rola 2 dados
    rolagemDeDados = ArremessaDados(2)

    #checa se são dados de valor repetido
    if(rolagemDeDados[0] != rolagemDeDados[1]):
      numeroRepetido = 0
      turno+=1

    else:
      numeroRepetido += 1
      if(numeroRepetido == 3):
        #caso seja preso, encerra o looping
        break
  return turno

Agora vem a parte de chamar essa função de cima incessantemente, somar os retornos e dividir pelo número de vezes em que a função foi chamada:

In [ ]:
def ChecaValorE(n):
  Y = 0
  for i in range(n):
    Y += JogaAteSerPreso()
  return Y/n

In [ ]:
ChecaValorE(10000)

214.9374

Fazendo alguns testes vemos que para n grandes o valor se aproxima de 216.



---

#d)

Uma justificativa seria tratando esse problema com uma distribuição gemétrica. Uma distribuição geométrica te diz qual a chance de ter um sucesso após n fracassos:

(1-p)^n * p

Seu valor esperado é justamente:

E[X] = p^-1

Em nosso contexto, fracasso seria não ser preso (curioso, né?) e sucesso ser preso.

Nesse caso a chance de ser preso é tirando números repetidos nos dados 3 vezes seguidas, mas como vimos na letra a, a chance de tirar 2 números iguais em dois dados é de 6/36, logo teríamos (6/36)(6/36)(6/36) como a probabilidade de ser preso.

E[X] = (6/36)^-3 = (1/6)^-3 = 216



---

#e)

Essa parte é legal. Pela definição de uma matriz de transição, sabemos que ela representa as probabilidades de dado que estou em estado X, a chance de eu ir para estado Y ou Z ou W, etc. Portanto, se pegarmos um vetor indicando onde começamos e começarmos a aplicar a matriz de transição nele, teremos novas probabilidades para cada casa, indicando quanto "tempo" se passa/passou nela ou qual a probabilidade de naquele instante eu estar nela.

Se quisermos começar na casa 1, teríamos um vetor com 123 entradas, sendo todas nulas e a 1° = 1, pois assim teríamos 100% de chance de estarmos na 1° casa. Aplicando a matriz de transição, vamos chamá-la de T, teremos então as probabilidades de, após o arremesso dos dados, estarmos em cada uma das 40 casas sabendo que estamos na casa 1 atualmente.

Seja v esse vetor com tudo zerado e 1 na 1° entrada:

v'T = vetor com as probabilidades de estarmos em cada uma das 40 casas sabendo que estamos agora na 1°.

Se aplicarmos novamente teremos (v'T)T ou v'(TT), pois matrizes e vetores são associativos. O que isso nos diz? O vetor resultante disso é um vetor com as probabilidades de estarmos em cada uma das 40 casas sabendo que estamos em alguma casa de v'T já descrita com suas probabilidades. Então conforme você vai multiplicando v por T e por T e assim em diante n vezes, estamos vendo na verdade qual a probabilidade de se estar em uma das casas do jogo **após** n rodadas.

Mas uma coisa interessante é que, conforme você aplica mais e mais a matriz T em v, você começa a convergir para algo, esse algo representa a probabilidade de se estar em uma das 40 casas em geral! Isso costuma funcionar para valores grandes como esse n->infinito, mas se não me engano nem sempre isso ocorre, porém infelizmente não tenho uma prova decente de quando ocorre ou não, mas nesse caso vamos cometer o erro de só supor que ocorre.

Mas isso é justamente a definição de função massa de probabilidade, não é? Dado valor x eu quero saber qual a probabilidade de X assumir ele. Em outras palavras queremos P(X = x) = pX(x). Portanto basta multiplicarmos v por uma quantidade grande de matrizes de transição T.

IMPORTANTE: Nesse contexto, a probabilidade de estar na casa 20 não diz nada sobre estar ou não preso, mas somente que o jogador está nela.



In [ ]:
def XnoTempoT(n, M):
  T = np.copy(M)
  E = T
  v = np.zeros(123)
  v[0] = 1

  if(n <= 0):
    return v
  if(n > 1):
    for i in range(n-1):
      E = E@T
  return v@E

Acima você tem uma função que aplica T n vezes.

Abaixo você pode escolher ver o gráfico com a probabilidade de se estar
em cada uma das 40 casas em um instante n. Basta definir n que seria o número de iterações feitas até o estado atual.

In [ ]:
n = 10

In [ ]:

v = XnoTempoT(n, M)
probs = list()
casas = list()

for i in range(40):
  casas.append(i+1)
  if(i != 19):
    probs.append(round((v[i] + v[i+40] + v[i+80])*100, 3))
  else:
    probs.append(round((v[i] + v[i+40] + v[i+80] + v[120] + v[121] + v[122])*100, 3))

fig = plt.figure(figsize = (10, 5))

# creating the bar plot
plt.bar(casas, probs, color ='blue',
        width = 0.4)

plt.xlabel("casa")
plt.ylabel("Probabilidade em %")
plt.title(f"Probabilidades por casa em {n} iterações")
plt.show()

NameError: ignored

Com n = 100 já temos uma aproximação interessante e podemos tratá-la como a função massa de probabilidade, pois temos a probabilidade de X assumir quaisquer valores naturais entre 1 e 40.

Isso tem uma ligação bem forte com autovalores e autovetores dessa matriz, mas infelizmente não consegui me aprofundar tanto nisso :(.

Dentre as coisas que li, destaco isso:
https://www.todasasconfiguracoes.com/2012/10/19/banco-imobiliario/

O restante foram discussões principalmente no https://math.stackexchange.com sobre autovetores e autovalores de matrizes estocásticas e um teorema que pode se relacionar a isso. Não me recordo mais os links certinhos, desculpe.
